浮点数格式：FP64, FP32, FP16, BFLOAT16, TF32之间的相互区别

https://www.cnblogs.com/lemonzhang/p/17843336.html

## 低精度训练

### fp16

In [ ]:
import torch
from transformers import TrainingArguments, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(..., torch_dtype=torch.half) # float16

args = TrainingArguments(
    ...,
    adam_epsilon = 1e-4,# 完全fp16时梯度爆炸可以设置这个
)

# 如果用了lora，新的模块会是FP32，一般没必要改到16
model = model.half()  # 当整个模型都是半精度时，需要将adam_epsilon调大


### bf16

In [ ]:

from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from peft import PeftModel, PeftConfig
import json

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "/home/zhuyao/Sunpeng/models/qwen_2B_instruct", torch_dtype=torch.int8, device_map="cuda"
)

# lora后可转换
# model = model.bfloat16()

In [ ]:
model.enable_input_require_grads

In [ ]:
for name, param in model.named_parameters():
    print(name,param.dtype)

## 量化

https://blog.csdn.net/qq_41956187/article/details/136997432

### 8bit  

https://www.cnblogs.com/LXP-Never/p/16822727.html

In [ ]:
from transformers import  BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 启用 8-bit 量化 [−128,127]
    llm_int8_threshold=6.0,  # 设置低精度计算的阈值：当某个权重的绝对值超过这个阈值时，认为它对模型性能的影响较大，将其保留为高精度类型。
    llm_int8_skip_modules=None  # 指定跳过量化的模块（可选）
)

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "/home/zhuyao/Sunpeng/models/qwen_2B_instruct", torch_dtype=torch.bfloat16, device_map="cuda", quantization_config=quantization_config,
)


### 4bit

https://huggingface.co/blog/zh/4bit-transformers-bitsandbytes

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 启用 4-bit 量化 [-8, 7]
    bnb_4bit_compute_dtype="float16",  # 计算时的精度:量化主要针对模型的权重存储，而实际计算仍然需要浮点数来保持精度。
    bnb_4bit_use_double_quant=True,  # 双重量化:4-bit 权重的值被先量化为 int8，然后再进一步压缩到 4-bit。
    bnb_4bit_quant_type="nf4"  # 量化类型（如 `nf4` 或标准量化） NF4 使用对数分布或其他非均匀分布来更精确地表示权重值。
)

#### QLora

在量化的模型上插入 LoRA 层，微调这些低秩参数。量化后的权重在反向传播时被临时去量化（恢复为高精度），以确保梯度计算的精度。
显存占用极低，支持超大模型。

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4",
)

model_name = "huggingface/llama-7b"
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
)

model = get_peft_model(model, lora_config)
